# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")

In [3]:
len(stocks)

505

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [4]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [5]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'

In [6]:
r = requests.get(api_url)
if r.status_code <200 and r.status_code>=300:
    raise Error(r.reason)
data = r.json()

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [7]:
price = data["latestPrice"]
marketCap = data["marketCap"]

In [8]:
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'DNTAESSBLOGQ EL/GRCA KM )TNL(ASEA',
 'calculationPrice': 'close',
 'open': None,
 'openTime': None,
 'openSource': 'olaificf',
 'close': None,
 'closeTime': None,
 'closeSource': 'ffilaioc',
 'high': None,
 'highTime': None,
 'highSource': None,
 'low': None,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 134.9,
 'latestSource': 'Close',
 'latestTime': 'December 24, 2020',
 'latestUpdate': 1661372105265,
 'latestVolume': None,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexLastUpdated': None,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 136.06,
 'previousVolume': 91688064,
 'change': 1.02,
 'changePercent': 0.00805,
 'volume': None,
 'iexMarketPercent': None,
 'iexVolume': None,
 'avgTotalVolume': 108041333,
 

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [9]:
my_columns = ["Ticker", "Stock Price", "Market Capitalization", "Number of Stocks to Buy"]
finalDataFrame = pd.DataFrame(columns=my_columns)

In [10]:
finalDataFrame.append(
    pd.Series(
    [
        symbol,
        data["latestPrice"],
        data["marketCap"],
        'N/A'
    ], index = my_columns
    ),
    ignore_index = True
)

,Ticker,Stock Price,Market Capitalization,Number of Stocks to Buy
0,AAPL,134.9,2284465987081,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [11]:
api_url_template = 'https://sandbox.iexapis.com/stable/stock/{0}/quote?token={1}'
my_columns = ["Ticker", "Stock Price", "Market Capitalization", "Number of Stocks to Buy"]
finalDataFrame = pd.DataFrame(columns=my_columns)

In [12]:
stocks["Ticker"][:5]

0       A
1     AAL
2     AAP
3    AAPL
4    ABBV
Name: Ticker, dtype: object

In [13]:
#Never run full scale in live environment
#Throttling would occur!!!
for stock in stocks["Ticker"][:5]:
        api_url = api_url_template.format(stock, IEX_CLOUD_API_TOKEN)
        r = requests.get(api_url)
        if r.status_code <200 or r.status_code>=300:
            raise Error(r.reason)
        data = r.json()
        finalDataFrame = finalDataFrame.append(pd.Series(
                [
                    data["symbol"],
                    data["latestPrice"],
                    data["marketCap"],
                    'N/A'
                ], 
                index = my_columns
            ),
            ignore_index = True)


In [14]:
finalDataFrame

,Ticker,Stock Price,Market Capitalization,Number of Stocks to Buy
0,A,117.54,36954012327,N/A
1,AAL,15.81,9875531919,N/A
2,AAP,166.34,11353397999,N/A
3,AAPL,134.01,2350293961018,N/A
4,ABBV,107.96,187924216077,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [15]:
from typing import List
def chunk(lst: List[str], n:int):
    #Yield successive n elements
    for i in range(0, len(lst), n):
        yield(lst[i:i+n])

In [16]:
symbol_strings = []
for stockSubList in chunk(stocks, 100):
    symbol_strings.append( ",".join(list(stockSubList["Ticker"])) )
my_columns = ["Ticker", "Stock Price", "Market Capitalization", "Number of Stocks to Buy"]
finalDataFrame = pd.DataFrame(columns=my_columns)
api_url_template = 'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={0}&types=quote&token={1}'
for stockstr in symbol_strings:
    r = requests.get(api_url_template.format(stockstr, IEX_CLOUD_API_TOKEN))
    if r.status_code <200 or r.status_code>=300:
        raise Error(r.reason)
    data = r.json()
    for value in  data.values():
        quoteData = value["quote"]
        finalDataFrame = finalDataFrame.append(pd.Series(
                [
                    quoteData["symbol"],
                    quoteData["latestPrice"],
                    quoteData["marketCap"],
                    'N/A'
                ], 
                index = my_columns
            ),
            ignore_index = True)

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [17]:
try:
    portfolio_size = float(input("Enter the size of your portfolio"))
except ValueError as e:
    print("Please enter a number")
except Error as e:
    print(e)

Enter the size of your portfolio100000000


In [18]:
position_size = portfolio_size / len(finalDataFrame)

In [19]:
for cnt in range(len(finalDataFrame)):
    finalDataFrame.iloc[cnt, -1] = math.floor(position_size / finalDataFrame.iloc[cnt, 1])
# finalDataFrame

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [20]:
worksheet_name = "Recommended trade to buy"
excel_writer = pd.ExcelWriter("recommendedStock.xlsx", engine = "xlsxwriter")
finalDataFrame.to_excel(excel_writer, worksheet_name, index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [21]:
background_color = "#0a0a23"
font_color = "#ffffff"

string_format = excel_writer.book.add_format(
    {
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)
dollar_format = excel_writer.book.add_format(
    {
        "num_format" : "$_(* #,##0.00_)",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)
integer_format = excel_writer.book.add_format(
    {
        "num_format": "0",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [22]:
column_formats = {
    "A": [my_columns[0], string_format],
    "B": [my_columns[1], dollar_format],
    "C": [my_columns[2], dollar_format],
    "D": [my_columns[3], integer_format]
}
for column in column_formats.keys():
    excel_writer.sheets[worksheet_name].set_column(
        "{0}:{0}".format(column),
        18,
        column_formats[column][1]
    )
    #Title
    excel_writer.sheets[worksheet_name].write(
        f"{column}1", column_formats[column][0], column_formats[column][1]
    )


## Saving Our Excel Output

Saving our Excel file is very easy:

In [23]:
excel_writer.save()

'Number of Stocks to Buy'